In [1]:
! nvidia-smi

Fri Jun  3 08:21:11 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!pip install pytorch-tabnet optuna 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 308 kB 8.4 MB/s 
     |████████████████████████████████| 209 kB 67.1 MB/s 
     |████████████████████████████████| 81 kB 10.4 MB/s 
     |████████████████████████████████| 78 kB 9.1 MB/s 
     |████████████████████████████████| 49 kB 6.9 MB/s 
     |████████████████████████████████| 112 kB 71.1 MB/s 
     |████████████████████████████████| 146 kB 64.5 MB/s 
  Created wheel for pyperclip: filename=pyperclip-1.8.2-py3-none-any.whl size=11137 sha256=5492a291352efe578a0e28f062a099fe45935e9f5eb6cd155800934258535b86
  Stored in directory: /root/.cache/pip/wheels/9f/18/84/8f69f8b08169c7bae2dde6bd7daf0c19fca8c8e500ee620a28
Successfully built pyperclip


In [3]:
import pandas as pd
import numpy as np
from pytorch_tabnet.tab_model import TabNetClassifier
from sklearn.metrics import accuracy_score,classification_report
import optuna as opt
import torch
import os
import joblib

In [4]:
def make_save_cv_model(i,model_name,model,best_params,optim,output_path="./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/cross_validated_models"):

    ''' This function saves cross validation model in the corresponding directory ( if the path does not exist it creates the path for it'''


    if os.path.exists(os.path.join(output_path,f"{i}_{model_name}_{optim}")):
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))
    else:
        os.mkdir(os.path.join(output_path,f"{i}_{model_name}_{optim}"))
        joblib.dump(model, os.path.join(output_path,f"{i}_{model_name}_{optim}/{i}_model.z"))
        with open(os.path.join(output_path,f"{i}_{model_name}_{optim}/model_params.txt"),"w+") as file:
            file.write(str(best_params))

In [5]:
def train(fold_dict,fold,model_name,sc_df,tar_col,optim,optim_trial,k_folds=10,tar_cols="",verbose=1):

    ''' this function is used to train the model with parameters optimization using optuna and cross validation using stratified k_folds'''

    y = sc_df[tar_col]
    x = sc_df.drop([tar_col],axis=1)
    model_name = model_name 
    def objective(trial):
      train_index = fold_dict[fold]["train"]
      test_index = fold_dict[fold]["test"]
      clf = TabNetClassifier(n_d=trial.suggest_int("n_d", 8, 64),
                              n_a =trial.suggest_int("n_a", 8, 64),
                              n_steps = trial.suggest_int("n_steps",3,10),
                              gamma =trial.suggest_float("gamma", 1.0, 2.0),
                              n_independent = trial.suggest_int("n_independent",1,5),
                              n_shared = trial.suggest_int("n_shared",1,5),
                              momentum = trial.suggest_float("momentum", 0.01, 0.4),
                              optimizer_fn = torch.optim.Adam,
                              # scheduler_fn = torch.optim.lr_scheduler,
                              # scheduler_params = {"gamma" :trial.suggest_float("sch-gamma", 0.5, 0.95), "step_size": trial.suggest_int("sch_step_size", 10, 20, 2)},
                              verbose = verbose,
                              device_name = "auto"
                              )
      # print(f" train_index :: {train_index}")
      # print(f" test_index :: {test_index}")
      X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
      # print(X_train.shape, X_test.shape)
      X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
      Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
      Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
      print(Y_train.shape, Y_test.shape)
      clf.fit(X_train, Y_train,
              eval_set=[(X_test, Y_test)],
              eval_metric=['accuracy'])
      Y_pred = clf.predict(X_test)
      print(classification_report(Y_test, Y_pred, labels=[x for x in range(6)]))
      acc = accuracy_score(Y_pred, Y_test)
      return acc

    print(f"Starting optimization for fold : [{fold}/{k_folds}]")
    study = opt.create_study(direction='maximize')
    study.optimize(objective, n_trials=optim_trial)
    best_params = study.best_params
    print(f" Best params for fold : [{fold}/{k_folds}]")
    print(best_params)
    joblib.dump(best_params,f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/comp/fold_{fold}_best_params.z")
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/best_params/fold_{fold}_best_params.txt", "w+") as file:file.write(str(best_params))
    print(f"Saved best_params at : outputs/{model_name}/best_params/fold_{fold}_best_params.txt")
    train_index = fold_dict[fold]["train"]
    test_index = fold_dict[fold]["test"]
    X_train,X_test = x.iloc[train_index,:], x.iloc[test_index,:]
    # print(X_train.shape, X_test.shape)
    X_train, X_test = X_train.to_numpy(dtype=np.float64), X_test.to_numpy(dtype=np.float64)
    Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]
    Y_train, Y_test = Y_train.to_numpy(dtype=np.float64), Y_test.to_numpy(dtype=np.float64)
    clf_model = TabNetClassifier(**study.best_params)
    clf_model.fit(X_train,Y_train)
    Y_pred = clf_model.predict(X_test)
    clf_report = classification_report(Y_test, Y_pred, labels=[x for x in range(6)])
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/classification_report/{model_name}_{fold}_classification_report.txt","w+") as file:file.write(str(clf_report))
    accuracy = accuracy_score(Y_pred, Y_test)
    with open(f"./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/{model_name}/{model_name}_{fold}_accuracy_score.txt","w+") as file:file.write(f" accuracy :: {str(accuracy)}")
    try:
        print("[++] Saving the model and parameters in corresponding directories")
        make_save_cv_model(fold,model_name,clf_model,best_params,optim=optim)
    except:
        print("[-] Failed to save the model")

In [6]:
use_df = pd.read_csv("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/outputs/data/trainable_scaled_balanced.csv")
tar_col = "PCE_categorical"
model_name = "pytorch_tabnet"
optimizer = "Adam"
fold_dict = joblib.load("./drive/MyDrive/SOLAR_CELL/ML_PROCESSED_DATA/inputs/fold_vals/fold_data.z")
fold = 5

In [7]:
train(fold_dict = fold_dict,
      fold = fold,
      model_name=model_name,
      sc_df=use_df,
      tar_col=tar_col,
      optim=optimizer,
      optim_trial = 15)
print(f"[++] Ended the training process for fold {fold}")

[I 2022-06-03 08:21:33,236] A new study created in memory with name: no-name-6e95cbc4-ebda-48f4-8b63-cf71dfbce2b8


Starting optimization for fold : [5/10]
Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.1681  | val_0_accuracy: 0.62522 |  0:00:02s
epoch 1  | loss: 0.82519 | val_0_accuracy: 0.62457 |  0:00:05s
epoch 2  | loss: 0.75707 | val_0_accuracy: 0.66041 |  0:00:07s
epoch 3  | loss: 0.72432 | val_0_accuracy: 0.65868 |  0:00:09s
epoch 4  | loss: 0.70752 | val_0_accuracy: 0.67552 |  0:00:12s
epoch 5  | loss: 0.68283 | val_0_accuracy: 0.6848  |  0:00:15s
epoch 6  | loss: 0.66955 | val_0_accuracy: 0.6943  |  0:00:17s
epoch 7  | loss: 0.66118 | val_0_accuracy: 0.68696 |  0:00:19s
epoch 8  | loss: 0.6519  | val_0_accuracy: 0.6997  |  0:00:22s
epoch 9  | loss: 0.65193 | val_0_accuracy: 0.70099 |  0:00:24s
epoch 10 | loss: 0.64958 | val_0_accuracy: 0.70877 |  0:00:26s
epoch 11 | loss: 0.64269 | val_0_accuracy: 0.70898 |  0:00:29s
epoch 12 | loss: 0.63373 | val_0_accuracy: 0.70531 |  0:00:31s
epoch 13 | loss: 0.64128 | val_0_accuracy: 0.70984 |  0:00:33s
epoch 14 | loss: 0.61919 | val_0_accuracy:

[I 2022-06-03 08:25:03,507] Trial 0 finished with value: 0.7841105354058722 and parameters: {'n_d': 55, 'n_a': 54, 'n_steps': 4, 'gamma': 1.18524595355649, 'n_independent': 1, 'n_shared': 3, 'momentum': 0.20621816805984905}. Best is trial 0 with value: 0.7841105354058722.


              precision    recall  f1-score   support

           0       0.98      1.00      0.99       826
           1       0.62      0.61      0.61       825
           2       0.80      0.91      0.85       822
           3       1.00      1.00      1.00       779
           4       0.57      0.54      0.55       761
           5       0.68      0.59      0.63       619

    accuracy                           0.78      4632
   macro avg       0.77      0.77      0.77      4632
weighted avg       0.78      0.78      0.78      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.95834 | val_0_accuracy: 0.55721 |  0:00:03s
epoch 1  | loss: 1.08431 | val_0_accuracy: 0.56779 |  0:00:07s
epoch 2  | loss: 0.99385 | val_0_accuracy: 0.58679 |  0:00:10s
epoch 3  | loss: 1.01784 | val_0_accuracy: 0.61248 |  0:00:14s
epoch 4  | loss: 0.87192 | val_0_accuracy: 0.60643 |  0:00:17s
epoch 5  | loss: 0.83007 | val_0_accuracy: 0.63752 |  0:00:21s
epoch 6  | loss: 0.79552 | val_0_accuracy: 

[I 2022-06-03 08:31:01,967] Trial 1 finished with value: 0.781951640759931 and parameters: {'n_d': 54, 'n_a': 14, 'n_steps': 9, 'gamma': 1.5510957863801187, 'n_independent': 2, 'n_shared': 1, 'momentum': 0.04310287994244824}. Best is trial 0 with value: 0.7841105354058722.


...

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.57      0.47      0.51       825
           2       0.71      0.91      0.80       822
           3       1.00      1.00      1.00       779
           4       0.52      0.49      0.50       761
           5       0.62      0.57      0.59       619

    accuracy                           0.75      4632
   macro avg       0.73      0.74      0.73      4632
weighted avg       0.74      0.75      0.74      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.64958 | val_0_accuracy: 0.50432 |  0:00:05s
epoch 1  | loss: 1.12246 | val_0_accuracy: 0.5408  |  0:00:10s
epoch 2  | loss: 0.98394 | val_0_accuracy: 0.56671 |  0:00:15s
epoch 3  | loss: 1.06621 | val_0_accuracy: 0.56261 |  0:00:20s
epoch 4  | loss: 0.93177 | val_0_accuracy: 0.5747  |  0:00:25s
epoch 5  | loss: 0.90773 | val_0_accuracy: 0.59218 |  0:00:31s
epoch 6  | loss: 0.87899 | val_0_accuracy: 

[I 2022-06-03 09:26:45,580] Trial 13 finished with value: 0.6623488773747841 and parameters: {'n_d': 33, 'n_a': 64, 'n_steps': 8, 'gamma': 1.4497866791446188, 'n_independent': 2, 'n_shared': 4, 'momentum': 0.13500092359188443}. Best is trial 2 with value: 0.8009499136442142.


              precision    recall  f1-score   support

           0       0.99      0.99      0.99       826
           1       0.40      0.32      0.36       825
           2       0.56      0.79      0.66       822
           3       1.00      1.00      1.00       779
           4       0.42      0.53      0.47       761
           5       0.61      0.25      0.36       619

    accuracy                           0.66      4632
   macro avg       0.66      0.65      0.64      4632
weighted avg       0.67      0.66      0.65      4632

Device used : cuda
(41688,) (4632,)
epoch 0  | loss: 1.49582 | val_0_accuracy: 0.56455 |  0:00:03s
epoch 1  | loss: 1.0224  | val_0_accuracy: 0.58225 |  0:00:06s
epoch 2  | loss: 0.86979 | val_0_accuracy: 0.61485 |  0:00:09s
epoch 3  | loss: 0.79153 | val_0_accuracy: 0.65371 |  0:00:12s
epoch 4  | loss: 0.74889 | val_0_accuracy: 0.67055 |  0:00:15s
epoch 5  | loss: 0.72788 | val_0_accuracy: 0.6617  |  0:00:19s
epoch 6  | loss: 0.7077  | val_0_accuracy: 

[I 2022-06-03 09:29:41,848] Trial 14 finished with value: 0.7687823834196891 and parameters: {'n_d': 24, 'n_a': 52, 'n_steps': 5, 'gamma': 1.8178616071133977, 'n_independent': 3, 'n_shared': 2, 'momentum': 0.33934668644101934}. Best is trial 2 with value: 0.8009499136442142.


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       826
           1       0.57      0.62      0.59       825
           2       0.79      0.91      0.84       822
           3       1.00      1.00      1.00       779
           4       0.53      0.50      0.51       761
           5       0.69      0.52      0.59       619

    accuracy                           0.77      4632
   macro avg       0.76      0.76      0.76      4632
weighted avg       0.77      0.77      0.77      4632

 Best params for fold : [5/10]
{'n_d': 42, 'n_a': 60, 'n_steps': 6, 'gamma': 1.575059266508576, 'n_independent': 3, 'n_shared': 3, 'momentum': 0.20431519971315895}
Saved best_params at : outputs/pytorch_tabnet/best_params/fold_5_best_params.txt
Device used : cuda
No early stopping will be performed, last training weights will be used.
epoch 0  | loss: 1.58986 |  0:00:04s
epoch 1  | loss: 1.0617  |  0:00:07s
epoch 2  | loss: 0.93178 |  0:00:11s
epoch 3  

Fold 0 has started running on 20-05-22 


Fold 0 has completed sucessfully on 17:00 20-05-22

Fold 1 has started running at 15:15 21-05-22

Fold 2 has started running at 09:45 22-05-22

Fold 2 has completed sucessfully on 10:58 22-05-22

Fold 3 has started running at 18:40 22-05-22

Fold 3 has completed sucessfully on 22-05-22

Fold 4 completed sucessfully on 21:04 on 22-05-22

Fold 5 started at 18:21 on 23-05-22

Fold 5 completed sucessfully on 19:44 on 23-05-22

Fold 6 started at 12:53 on 24-05-22

Fold 6 has completed at 14:14 on 24-05-22

Fold 7 started at 14:18 on 24-05-22

Fold 7 execution failed due to colab gpu time limit

Fold 7 trial 1 started at 11:00 on 25-05-22

Fold 7 has completed sucessfully at 12:14 on 25-05-22 

Fold 8 has started at 9:38 on 26-05-22

Fold 8 filed due to interrupted internet connection

Fold 8 trial 1 started at 13:38 on 26-05-22

Fold 8 has successfully executed at 15:33 on 26-05-22

Fold 9 has started at 13:35 on 27-05-22

Fold 9 has completed at 14:55 on 27-05-22

Editing with rectified dataset witout duplicacy because of space values

Fold 0 started at 13:21 on 28-05-22

Fold 0 completed sucessfully at 14:46 on 28-05-22

Fold 1 Failed to run due to some index error

_____ Restarting the training process again due to data distribution failure____



Fold 0 started at 10:47 on 30-05-22

Fold 0 completed successfully at 12:30 on 30-05-22

Fold 1 started at 8:27 on 31-05-22

Fold 1 execution failed due to runtime disconnection

Fold 1 started again at 9:38 on 31-05-22

Fold 1 execution failed due to gpu disconnect 

Fold 1 started again at 8:36 on 1-06-22

Fold 1 execution failed due to network disconnection

Fold 1 started again at 13:11 on 01-06-22

Fold 1 has succesfully executed at 14:25 on 01-06-22

Fold 2 started at 14:29 on 01-06-22

Fold 2 completed succesfully at 16:00 on 01-06-22

Fold 3 started at 09:43 on 02-06-22

Fold 3 execution failed due to gpu server disconnection 

Fold 3 started again at 13:34 on 02-06-22

Fold 3 ran successfully at 14:47 on 02-06-22

Fold 4 started at 14:48 on 02-06-22

Fold 4 ran successfully at 04:01 on 02-06-22

Fold 5 started at 13:51 on 03-06-22